# Архитектура RNN. Классификация текста.

Привет! В это семинаре мы познакомимся с задачей классификации текста, на примере поиска тематики новости, а также с двумя из основных архитектур рекуррентных нейросетей – RNN и GRU.

Нам потребуется одна библиотека от `HuggingFace🤗` под названием `datasets`. Она содержит большое число датасетов, которые используются в NLP.

In [4]:
#!pip install datasets

In [1]:
import random
import numpy as np

import nltk
import gensim.downloader as api

import torch
import torch.nn as nn
import datasets

In [2]:
# За детерминизм!
SEED = 0xDEAD
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)

Загрузим датасет новостей: `AgNews`. В нем разделены тексты на 4 темы: `World`, `Sports`, `Business`, `Sci/Tech`. Посмотрим на структуру датасета и на примеры текстов:

In [3]:
dataset = datasets.load_dataset("ag_news")
dataset["train"]

Using custom data configuration default
Reusing dataset ag_news (C:\Users\BIT\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [9]:
dataset["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

В `dataset` находятся `train` и `test` части датасета.

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

Чтобы превращать текст из набора слов в набор векторов мы будем использовать предобученные эмбеддинги. Посмотрим на их список и выберем один из них.

In [6]:
print("\n".join(api.info()['models'].keys()))

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [7]:
word2vec = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


Токенезируем наш текст с помощью NLTK.

In [10]:
MAX_LENGTH=128

tokenizer = nltk.WordPunctTokenizer()

dataset = dataset.map( # обращаемся к dataset (train и test)
    lambda item: {
        "tokenized": tokenizer.tokenize(item["text"])[:MAX_LENGTH] # создаём tokenized для каждого словаря (text, label)
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'tokenized'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'tokenized'],
        num_rows: 7600
    })
})

In [16]:
dataset['train']['tokenized'][0]

['Wall',
 'St',
 '.',
 'Bears',
 'Claw',
 'Back',
 'Into',
 'the',
 'Black',
 '(',
 'Reuters',
 ')',
 'Reuters',
 '-',
 'Short',
 '-',
 'sellers',
 ',',
 'Wall',
 'Street',
 "'",
 's',
 'dwindling',
 '\\',
 'band',
 'of',
 'ultra',
 '-',
 'cynics',
 ',',
 'are',
 'seeing',
 'green',
 'again',
 '.']

Создадим мапинг из токенов в индексы

In [17]:
word2idx = {word: idx for idx, word in enumerate(word2vec.index2word)} # ключ - слово, значение - индекс
# word2vec.index2word - по индексам возвращает слово (лист слов)
# word2idx - по слову возвращает индекс

In [20]:
word2idx

{'<user>': 0,
 '.': 1,
 ':': 2,
 'rt': 3,
 ',': 4,
 '<repeat>': 5,
 '<hashtag>': 6,
 '<number>': 7,
 '<url>': 8,
 '!': 9,
 'i': 10,
 'a': 11,
 '"': 12,
 'the': 13,
 '?': 14,
 'you': 15,
 'to': 16,
 '(': 17,
 '<allcaps>': 18,
 '<elong>': 19,
 ')': 20,
 'me': 21,
 'de': 22,
 '<smile>': 23,
 '！': 24,
 'que': 25,
 'and': 26,
 '。': 27,
 '-': 28,
 'my': 29,
 'no': 30,
 '、': 31,
 'is': 32,
 'it': 33,
 '…': 34,
 'in': 35,
 'n': 36,
 'for': 37,
 '/': 38,
 'of': 39,
 'la': 40,
 "'s": 41,
 '*': 42,
 'do': 43,
 "n't": 44,
 'that': 45,
 'on': 46,
 'y': 47,
 "'": 48,
 'e': 49,
 'o': 50,
 'u': 51,
 'en': 52,
 'this': 53,
 'el': 54,
 'so': 55,
 'be': 56,
 "'m": 57,
 'with': 58,
 'just': 59,
 '>': 60,
 'your': 61,
 '^': 62,
 'like': 63,
 'have': 64,
 'te': 65,
 'at': 66,
 '？': 67,
 'love': 68,
 'se': 69,
 'are': 70,
 '<': 71,
 'm': 72,
 'r': 73,
 'if': 74,
 'all': 75,
 'b': 76,
 '・': 77,
 'not': 78,
 'but': 79,
 'we': 80,
 'es': 81,
 'ya': 82,
 '&': 83,
 'follow': 84,
 'up': 85,
 'what': 86,
 'get': 87

Переведем токены в индексы

In [26]:
def encode(word):
    if word in word2idx.keys():
        return word2idx[word]
    return word2idx["unk"]

In [27]:
dataset = dataset.map(
    lambda item: { # item элемент словаря из словаря {'train':...,'test':...}
        "features": [encode(word) for word in item["tokenized"]] # заменяем набор текста из каждого item['features'] на лист индексов по item['tokenized']
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [28]:
dataset["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'tokenized': ['Wall',
  'St',
  '.',
  'Bears',
  'Claw',
  'Back',
  'Into',
  'the',
  'Black',
  '(',
  'Reuters',
  ')',
  'Reuters',
  '-',
  'Short',
  '-',
  'sellers',
  ',',
  'Wall',
  'Street',
  "'",
  's',
  'dwindling',
  '\\',
  'band',
  'of',
  'ultra',
  '-',
  'cynics',
  ',',
  'are',
  'seeing',
  'green',
  'again',
  '.'],
 'features': [62980,
  62980,
  1,
  62980,
  62980,
  62980,
  62980,
  13,
  62980,
  17,
  62980,
  20,
  62980,
  28,
  62980,
  28,
  49286,
  4,
  62980,
  62980,
  48,
  137,
  214902,
  370,
  1645,
  39,
  8606,
  28,
  380053,
  4,
  70,
  1321,
  1745,
  389,
  1]}

In [29]:
dataset.remove_columns_(["text", "tokenized"]) # так как у нас есть фичи (индексы векторов word2vec), удаляем текст

C:\Users\BIT\AppData\Local\Temp/ipykernel_7068/3782232894.py:1: FutureWarning: remove_columns_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.remove_columns instead.
  dataset.remove_columns_(["text", "tokenized"])


Переведем в тензоры

In [31]:
dataset.set_format(type='torch')

In [32]:
dataset["train"][0]

{'label': tensor(2),
 'features': tensor([ 62980,  62980,      1,  62980,  62980,  62980,  62980,     13,  62980,
             17,  62980,     20,  62980,     28,  62980,     28,  49286,      4,
          62980,  62980,     48,    137, 214902,    370,   1645,     39,   8606,
             28, 380053,      4,     70,   1321,   1745,    389,      1])}

Хотим склеить объекты разной длинны в батчи. Для этого давайте напишем `collate_fn`.

In [33]:
def collate_fn(batch): # torch.long для дискретных матриц
    max_len = max(len(row["features"]) for row in batch) # для одинакового предствления данных в батче
    input_embeds = torch.empty((len(batch), max_len), dtype=torch.long) # матрица длина батча x максимальную длину (batch_size x seq_len)
    labels = torch.empty(len(batch), dtype=torch.long) # вектор лейблов
    for idx, row in enumerate(batch): # заполняем input_embeds данными; row словарарь(features, labels) (элемент датасета)
        to_pad = max_len - len(row["features"]) # кол-во элементов для допаддинга
        input_embeds[idx] = torch.cat((row["features"], torch.zeros(to_pad))) # в соответ. по индексу строку матрицы input_embeds вставляем конкат. эмбеддинга и нулей доппадинга (строка - слово) 
        labels[idx] = row["label"] # в соответсвующий по индексу элемент вектрора лейблов вствляем лейбл
    return {"features": input_embeds, "labels": labels}

In [53]:
from torch.utils.data import DataLoader

loaders = { # словарь('train':train_loader,'test':test_loader)
    k: DataLoader(
        ds, shuffle=(k=="train"), batch_size=32, collate_fn=collate_fn
    ) for k, ds in dataset.items()
}# k - train,test; ds - словарь(features,labels)

## CNN

Первая модель, которую мы рассмотрим: CNN. Одномерная конволюция достаточно хорошо справляется с задачей классификации. В конце надо собрать вектор текста с помощью `AdaptiveMaxPool1d` или `AdaptiveAvgPool1d`. Для классиффикации можно собрать любую Feed Forward Network.

In [58]:
!nvidia-smi

Sat Oct  9 13:07:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.12       Driver Version: 472.12       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   44C    P8     2W /  N/A |   2657MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [38]:
class CNNModel(nn.Module): # делаем свёртки на векторах
    def __init__(self, embed_size, hidden_size, num_classes=4): # может быть слова из 3 слов содержат достаточную информацию для классификации текста (например, по токсичности (3 слова грубых))
        super().__init__()
        self.embeddings = nn.Embedding(len(word2idx), embedding_dim=embed_size) # 1193514 x 25
        self.cnn = nn.Sequential(
            nn.Conv1d(embed_size, hidden_size, kernel_size=3, padding=1, stride=2), # embed_size и hidden_size отвечают за размер столбцов(размер эмбеддингов слов предложения)
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1), # берёт максимум по всем каналам, изменяя только последний канал (оставляет один столбец - эмбеддинг предложения) (batch_size x hidden_size (50) x 1)
            nn.Flatten(),
        ) nn.AdaptiveMaxPool1d
        self.cl = nn.Sequential(
            nn.Linear(hidden_size, num_classes) # (batch_size x hidden_size x 1) -> (batch_size x num_classes)
        )

    def forward(self, x): # x - batch_size x seq_len (строки-предложения, которые являются списками индексов)
        x = self.embeddings(x)  # подаются индексы получаются эмбеддинги (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1) # меняет размерности в соответсвующем порядке цифр (batch_size, embed_dim, seq_le); x - батч столбцов-эмбеддингов соответсвующих слов в предложении
        x = self.cnn(x) #Conv-BN-RELU+Conv-BN-RELU+Conv-BN-RELU+AMaxPool
        prediction = self.cl(x) # batch_size x num_classes
        return prediction 

In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel(word2vec.vector_size, 50).to(device) # 50 - hidden_size - на сколько каналов будем расширять эмбеддинги в свёрктах
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1

Подготовим функцию для обучения модели:

In [43]:

from tqdm.notebook import tqdm, trange


def training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm=2):
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False)
        for batch in pbar:
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum() # argmax(-1) - (batch_size x num_classes -> batch_size x 1 (номер столбца))
                num_objs += len(labels)
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs}")

In [54]:
training(model, criterion, optimizer, num_epochs, loaders)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

Valid Loss: 0.4036913812160492, accuracy: 0.8590788841247559


## RNN

Вторая модель: RNN. Это рекуррентная сеть, она использует скрытое состояние из прошлой иттерации для создания нового. Это описывается с помощью формул:

$$
h_t = \tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh})
$$

Напишем этот модуль на `Torch`!



In [121]:
class RNN(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_h = nn.Parameter(torch.rand(hidden_size, hidden_size)) # W.T
        self.b_h = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_x = nn.Parameter(torch.rand(embed_size, hidden_size)) # V.T # (batch_size x embed_size * embed_size x hidden_size -> batch_size x hidden_size)
        self.b_x = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden=None):
        '''
        x – torch.FloatTensor with the shape (bs, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device) # (batch_size x hidden_size) [hidden - строка, в тетради h был столбцом]
        seq_length = x.size(1) 
        for cur_idx in range(seq_length):
            hidden = torch.tanh( # hidden задан реккурентно
                x[:, cur_idx] @ self.w_x + self.b_x + hidden @ self.w_h + self.b_h # x[:,cur_idx] берём эмбеддинг слова из предложения -> (batch_size x embed_dim)
            ) # x[:, cur_idx] @ self.w_x -- (batch_size x embed_size * embed_size x hidden_size -> batch_size x hidden_size)
        return hidden

In [122]:
class RNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embeddings = nn.Embedding(len(word2idx), embed_size)
        self.rnn = RNN(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes) # (batch_size x hidden_size) -> (batch_size x num_classes) 

    def forward(self, x):
        x = self.embeddings(x) # (batch_size,seq_len, embed_dim)
        hidden = self.rnn(x)
        output = self.cls(hidden)
        return output

In [123]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = RNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [124]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

Valid Loss: 1.4723517894744873, accuracy: 0.2510526180267334


## GRU

Третья модель: GRU. Она усложненная версия `RNN`. Гланая идея GRU: гейты. Так реализуется "память" модели – она маскирует часть старого скрытого состояния, создавая на этом месте новое. Модель GRU описывается следующим образом:

$$
\begin{array}{ll}
            r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
            z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
            n_t = \tanh(W_{in} x_t + b_{in} + r_t * (W_{hn} h_{(t-1)}+ b_{hn})) \\
            h_t = (1 - z_t) * n_t + z_t * h_{(t-1)}
        \end{array}
$$

In [125]:
class GRU(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size
# параметры для r
        self.w_rh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_rh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_rx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_rx = nn.Parameter(torch.rand(1, hidden_size))
# параметры для z
        self.w_zh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_zh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_zx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_zx = nn.Parameter(torch.rand(1, hidden_size))
# параметры для n
        self.w_nh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_nh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_nx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_nx = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden = None):
        '''
        x – torch.FloatTensor with the shape (bs, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device) # (batch_size,hidden_size) [строка для каждого батча]
        
        for cur_idx in range(x.size(1)):
            r = torch.sigmoid(
                x[:, cur_idx] @ self.w_rx + self.b_rx + hidden @ self.w_rh + self.b_rh
            )
            z = torch.sigmoid(
                x[:, cur_idx] @ self.w_zx + self.b_zx + hidden @ self.w_zh + self.b_zh
            )
            n = torch.tanh(
                x[:, cur_idx] @ self.w_nx + self.b_nx + r * (hidden @ self.w_nh + self.b_nh)
            )
            hidden = (1 - z) * n + z * hidden

        return hidden

In [126]:
class GRUModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embed = nn.Embedding(len(word2idx), embed_size)
        self.gru = GRU(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embed(x)
        hidden = self.gru(x)
        output = self.cls(hidden)
        return output

In [127]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [128]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

Valid Loss: 0.8999207019805908, accuracy: 0.6455262899398804


## GRU + Embeddings

Мы не просто так загрузили эмбэдинги в начале. Давай использовать их вместо случайной инициализации! Для этого надо немного переделать способ подачи данных в модель и добавить в модель модуль `Embedding`. По-экспериментируем на модели `GRU`.

In [135]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [133]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word)) # на соответвующую строку (слово) по индексу ставим вектор эмбеддингов (этого слова)

C:\Users\BIT\AppData\Local\Temp/ipykernel_7068/1175556816.py:4: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  model.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word))


In [134]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

Valid Loss: 0.4243246912956238, accuracy: 0.8534210324287415


Попробуем заморозить эмбединги на первых итерациях обучения. Это поможет не сильно портить наши эмбединги на первых итерациях.

In [136]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embed
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

In [137]:
def training_freeze(model, criterion, optimizer, num_epochs, loaders, max_grad_norm=2, num_freeze_iter=1000):
    freeze_embeddings(model)
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False)
        for batch in pbar:
            if num_iter > num_freeze_iter and e < 1:
                freeze_embeddings(model, True)
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_objs += len(labels)
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs}")

In [138]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [139]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [140]:
training_freeze(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

Valid Loss: 0.46335598826408386, accuracy: 0.8368420600891113


Обычно с заморозкой качество получается лучше, но не в этот раз 

Эмбеддинги можно дообучать для того, чтобы разные по смыслу слова, но встречающиеся вместе часто, имели схожие эмбеддинги